# Example 1: Constructing the reduced Hessian

In this example, we will solve a model and check the eigenvalues of the reduced Hessian at the solution. The purpose of this example is to (a) practice using `PyomoNLP` and (b) introduce you to the reduced Hessian, which can be useful for debugging regularization coefficients.

We start with some imports.

In [2]:
import pyomo.environ as pyo
import pyomo.environ as pyo
from pyomo.common.collections import ComponentSet
from pyomo.contrib.incidence_analysis import IncidenceGraphInterface
from pyomo.contrib.pynumero.interfaces.pyomo_nlp import PyomoNLP
from svi.auto_thermal_reformer.fullspace_flowsheet import make_optimization_model
import numpy as np
import scipy as sp

The model we will solve comes from the "surrogate-vs-implicit" repository, available [here](https://github.com/robbybp/surrogate-vs-implicit). We have imported this package above, and will construct an autothermal reforming flowsheet model using its function `make_optimization_model`.

In [3]:
def make_and_solve_model(**kwds):
    # First, we make our model
    P = kwds.pop("P", 1.5e6)
    # With X = 0.95, we converge. With X = 0.94, we don't. Go figure
    X = kwds.pop("X", 0.94)
    m = make_optimization_model(X, P)
    m.ipopt_zL_out = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    m.ipopt_zU_out = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT_EXPORT)
    # Does the model solve?
    solver = pyo.SolverFactory("cyipopt", options=kwds)
    res = solver.solve(m, tee=True)
    # Converges infeasible in 900 iter
    return m, res

In [4]:
m, res = make_and_solve_model(X=0.95)
pyo.assert_optimal_termination(res)

2025-05-06 22:16:36 [INFO] idaes.init.fs.reformer.control_volume.properties_in: Starting initialization
2025-05-06 22:16:36 [INFO] idaes.init.fs.reformer.control_volume.properties_in: Property initialization: optimal - Optimal Solution Found.
2025-05-06 22:16:36 [INFO] idaes.init.fs.reformer.control_volume.properties_out: Starting initialization
2025-05-06 22:16:36 [INFO] idaes.init.fs.reformer.control_volume.properties_out: Property initialization: optimal - Optimal Solution Found.
2025-05-06 22:16:36 [INFO] idaes.init.fs.reformer.control_volume: Initialization Complete
2025-05-06 22:16:37 [INFO] idaes.init.fs.reformer: Initialization Complete: optimal - Optimal Solution Found
2025-05-06 22:16:37 [INFO] idaes.init.fs.reformer_recuperator.hot_side.properties_in: Starting initialization
2025-05-06 22:16:37 [INFO] idaes.init.fs.reformer_recuperator.hot_side.properties_in: Property initialization: optimal - Optimal Solution Found.
2025-05-06 22:16:37 [INFO] idaes.init.fs.reformer_recupera

2025-05-06 22:16:39 [INFO] idaes.init.fs.air_compressor_s2.control_volume.properties_out: Starting initialization
2025-05-06 22:16:39 [INFO] idaes.init.fs.air_compressor_s2.control_volume.properties_out: Property initialization: optimal - Optimal Solution Found.
2025-05-06 22:16:39 [INFO] idaes.init.fs.air_compressor_s2.control_volume.properties_out: Property package initialization: optimal - Optimal Solution Found.
2025-05-06 22:16:39 [INFO] idaes.init.fs.air_compressor_s2.properties_isentropic: Starting initialization
2025-05-06 22:16:39 [INFO] idaes.init.fs.air_compressor_s2.properties_isentropic: Property initialization: optimal - Optimal Solution Found.
2025-05-06 22:16:39 [INFO] idaes.init.fs.air_compressor_s2.properties_isentropic: Property package initialization: optimal - Optimal Solution Found.
2025-05-06 22:16:39 [INFO] idaes.init.fs.air_compressor_s2: Initialization Complete: optimal - Optimal Solution Found
2025-05-06 22:16:39 [INFO] idaes.init.fs.intercooler_s2.control_vo

Now we'll construct a `PyomoNLP` at the solution. This will give us access to the derivative matrices we'll need to construct the reduced Hessian.

In [5]:
nlp = PyomoNLP(m)

The reduced Hessian relies on knowledge of our "degrees of freedom." We'll assume we know what these are.

In [6]:
dof_varnames = [
    "fs.reformer_bypass.split_fraction[0.0,bypass_outlet]",
    "fs.reformer_mix.steam_inlet_state[0.0].flow_mol",
    "fs.feed.properties[0.0].flow_mol",
]
dofvars = [m.find_component(name) for name in dof_varnames]
dofvar_set = ComponentSet(dofvars)
basicvars = [v for v in nlp.get_pyomo_variables() if v not in dofvar_set]
print(f"N. dof variables:   {len(dofvars)}")
print(f"N. basic variables: {len(basicvars)}")

N. dof variables:   3
N. basic variables: 895


We need the Jacobian of our equality constraints with respect to "basic variables" to be nonsingular.

In [10]:
eqcons = nlp.get_pyomo_equality_constraints()
print(f"N. equality constraints: {len(eqcons)}")
basic_submatrix = nlp.extract_submatrix_jacobian(basicvars, eqcons)
try:
    lu = sp.sparse.linalg.splu(basic_submatrix.tocsc())
    print(f"Eq. Jac. has rank at least {len(basicvars)}")
except RuntimeError:
    # This matrix being singular doesn't prove anything about the rank.
    # We could have just chosen a bad set of degrees of freedom.
    print("Eq. Jac. is not (necessarily) full row rank")

N. equality constraints: 895
Eq. Jac. has rank at least 895


This proves that our equality Jacobian is full row rank. Now we can start worrying about the reduced Hessian.
When constructing anything involving the Hessian of the Lagrangian, it's always a good idea to construct the
gradient of the Lagrangian and make sure we have the right sign convention.

In [12]:
def get_gradient_of_lagrangian(
    nlp,
    primal_lb_multipliers,
    primal_ub_multipliers,
):
    grad_obj = nlp.evaluate_grad_objective()
    jac = nlp.evaluate_jacobian()
    duals = nlp.get_duals()
    conjac_term = jac.transpose().dot(duals)
    grad_lag = (
        - grad_obj
        - conjac_term
        + primal_lb_multipliers
        - primal_ub_multipliers
    )
    return grad_lag

lbmult = [m.ipopt_zL_out[x] for x in nlp.get_pyomo_variables()]
ubmult = [m.ipopt_zU_out[x] for x in nlp.get_pyomo_variables()]
gradlag = get_gradient_of_lagrangian(nlp, lbmult, ubmult)
print(max(abs(gradlag)))

9.090909151587457e-15


Looks good. Now we can start working on the reduced Hessian. The reduced Hessian is the Hessian of the Lagrangian projected onto the null space of the equality constraint Jacobian. We'll start by constructing the Hessian of the Lagrangian, with variables in the order `(dofvars, basicvars)`.